## Das Optimierungsproblem von Standorten

Es geht um die Auswahl von Standorten, um Kunden gut zu betreuen. Für Logistikzentren, für Nahversorgung, Planung von Ärztezentren etc.

Konzeptionell:
- Kundendistanz max. 50km zu Standort
- mind. 90% Kunden abdecken
- Bevorzugung der Top200 Städte in Deutschland (Potenzielle Kundschaft)
- Bevorzugung aktuelle Kundschaft der Stadt (Aktuelle Kundschaft)
- Minimieren der Standorte als Ziel

Im folgenden werden genutzt:
- der Optimierer PulP. Er ist Industrie Standard und Open Source. 
- PGeoCode zur Identifizierung von Geo-Coordinaten auf Grundlage von PLZ-Codes

Optimierung funktioniert über Lineare Programmierung
- Zielfunktion ist Kostenminimierung
- Kosten entstehen durch Standorte
- Nebenbedingung beschreibt, wie die Kunden abgedeckt sein müssen (Beispiel 1: jeder Kunde mind. 1 Standort. Beispiel 2: mind. 90% der Kunden mit mind. 1 Standort)
- Standortdistanz max. 50km zu Kundenort
- Kosten-Malus für Nebenstädte, Bevorzugung der Top 200 Städte Deutschlands

Note:
- die PLZ der candidates (potenzielle Standorte) sollten die zentralen PLZ der Stadt sein, damit die Abstände zu Kunden und anderen Städten korrekt berechnet werden

### Einfache Einsteigerfreundliche Version

In [27]:
import pandas as pd
import numpy as np
import pulp
import pgeocode

# --- KONFIGURATION ---
MAX_DISTANCE_KM = 50.0  # Harte Grenze (ggf. auf 38 reduzieren für "echte Fahrstrecke")
METROPOLIS_BONUS = 0.8  # Kostenfaktor für Top-Städte (niedriger = bevorzugt)
STANDARD_COST = 1.0     # Kostenfaktor für normale Standorte

# ---------------------------------------------------------
# 1. MOCK DATEN ERSTELLEN (Simulation deiner Ausgangslage)
# ---------------------------------------------------------
print("1. Erstelle Mock-Daten...")

# Simulierte Top Städte (Beispiel Rhein-Main)
top_cities_data = {
    'city': ['Frankfurt am Main', 'München', 'Hamburg', 'Köln', 'Mainz', 'Wiesbaden'],
    'plz': ['60311', '80331', '20095', '50667', '55116', '65185'],
    'is_top_200': [True, True, True, True, True, True]
}
df_candidates = pd.DataFrame(top_cities_data)

# Simulierte Kunden (Aggregiert auf PLZ5)
# Hanau (nahe FFM), Offenbach (nahe FFM), Rüsselsheim (Mitte), Fulda (weit weg)
customer_data = {
    'city': ['Hanau', 'Offenbach', 'Rüsselsheim', 'Fulda', 'Frankfurt am Mainz'],
    'plz5': ['63450', '63065', '65428', '36037', '60311'], # Hanau, Offenbach, Rüsselsheim, Fulda, FFM
    'customer_count': [500, 800, 400, 200, 1000]
}
df_demand = pd.DataFrame(customer_data)

# Hinzufügen Kunden-Standorte als potenzielle Kandidaten und De-Duplizieren (falls schon in top200 liste)
additional_candidates = pd.DataFrame({
    'city': customer_data['city'],
    'plz': customer_data['plz5'],
    'is_top_200': [False, False, False, False, True]
})
df_candidates = pd.concat([df_candidates, additional_candidates], ignore_index=True).drop_duplicates(subset=['plz'])

# ---------------------------------------------------------
# 2. GEOCODING (PLZ -> Lat/Lon)
# ---------------------------------------------------------
print("2. Geocoding der Standorte...")
nomi = pgeocode.Nominatim('de')

def get_coords(plz_series):
    # pgeocode query_postal_code erwartet Strings
    geo = nomi.query_postal_code(plz_series.astype(str).tolist())
    return geo[['latitude', 'longitude']].values

# Koordinaten für Kandidaten und Demand Points holen
df_candidates[['lat', 'lon']] = get_coords(df_candidates['plz'])
df_demand[['lat', 'lon']] = get_coords(df_demand['plz5'])

# Bereinigen von ungültigen PLZs (falls pgeocode nichts findet)
df_candidates.dropna(subset=['lat'], inplace=True)
df_demand.dropna(subset=['lat'], inplace=True)

# ---------------------------------------------------------
# 3. DISTANZ-MATRIX & PRE-FILTERING
# ---------------------------------------------------------
print("3. Berechne Distanzen und Coverage-Logik...")

def haversine_vectorized(lat1, lon1, lat2, lon2):
    # Vektorisierte Haversine Formel für Performance
    R = 6371  # Erdradius in km
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    dphi = np.radians(lat2 - lat1)
    dlambda = np.radians(lon2 - lon1)
    a = np.sin(dphi/2)**2 + np.cos(phi1)*np.cos(phi2) * np.sin(dlambda/2)**2
    return 2 * R * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

# Dictionary: Demand_PLZ -> Liste [Candidate_Index, Candidate_Index, ...]
# Nur Kandidaten < 50km werden hier eingetragen!
coverage_map = {} 
uncoverable_demand = []

# koordinaten in numpy arrays für vektorisierte Berechnung überführen, weil schneller
candidates_lat = df_candidates['lat'].values
candidates_lon = df_candidates['lon'].values

# Für jeden Demand Point die Distanzen zu allen Kandidaten berechnen und standorte <= 50km speichern
for idx, row in df_demand.iterrows():
    # Distanz von DIESEM Kunden zu ALLEN Kandidaten
    dists = haversine_vectorized(row['lat'], row['lon'], candidates_lat, candidates_lon)
    
    # Filter: Wer ist nah genug?
    # Note: np.where gibt np array zurück, wir holen uns nur die Indices die liste
    valid_indices = np.where(dists <= MAX_DISTANCE_KM)[0]

    # Note: Falls alle Kundenstandorte auch den Candidate_Standorten hinzugefügt wurden,
    # macht diese Überprüfung keinen Sinn mehr, da jeder Standort sich selbst abdeckt.
    # Evtl. wollen wir später Kundenstandorte ausschließen. Also lassen ich die Überprüfung drin.
    if len(valid_indices) > 0:
        coverage_map[idx] = valid_indices
    else:
        uncoverable_demand.append(row['plz5'])

print(f"-> {len(uncoverable_demand)} PLZ-Gebiete können nicht abgedeckt werden (kein Standort in {MAX_DISTANCE_KM}km).")

# ---------------------------------------------------------
# 4. OPTIMIERUNG (PuLP Solver)
# ---------------------------------------------------------
print("4. Starte Solver...")

# LP Problem initialisieren. Ziel: Minimiere Kosten
prob = pulp.LpProblem("Facility_Location_Optimization", pulp.LpMinimize)

# Entscheidungsvariablen: Welche Candidate Site machen wir auf? (kann für jeden stadt_index 0 oder 1 annehmen für nein, ja)
# der solver wird diese variablen so setzen, dass die Zielfunktion minimiert wird unter einhaltung der constraints
candidate_indices = df_candidates.index.tolist()
location_vars = pulp.LpVariable.dicts("Open_Site", candidate_indices, 0, 1, pulp.LpBinary)

# --- ZIELFUNKTION ---
# Wir minimieren die Kosten, dabei...
# ...bevorzugen wir TOP200 Städten (wegen Kundenpotenzial)
# ...bevorzugen wir Städte mit vielen Kunden (aktuelle Kundschaft) 

# Die Gesamtkosten sind die Summe aus den Kosten die für jede Stadt entstehen
# Also erstellen wir hier eine Liste mit den Kosten pro Stadt 
# und summieren diese dann in der Zielfunktion
costs = []
for idx in candidate_indices: 
    cost_factor = (
        METROPOLIS_BONUS 
        if df_candidates.at[idx, 'is_top_200'] # is top_city
        else STANDARD_COST
    )
    # Nachfrage-Bonus: Wenn am Standort Nachfrage ist, mache die Kosten niedriger 
    demand_here = (
        df_demand[df_demand['plz5'] == df_candidates.at[idx, 'plz']]
        ['customer_count'].sum()
    )
    if demand_here > 0:
        cost_factor = cost_factor / np.sqrt(demand_here)  
        # Niedrigere Kosten für hohe Nachfrage 
        # skaliere demand_here um den Effekt anzupassen (e.g. sqrt(demand))
    
    costs.append(location_vars[idx] * cost_factor) # locations_vars[idx] ist 0 oder 1 und eine Solver-Variable

# Zielfunktion setzen. Merke: pulp.lpSum fügt alle Kostenfunktionen pro Stadt zusammen. 
prob += pulp.lpSum(costs)

# --- CONSTRAINTS ---
# Jeder abdeckbare Kunde MUSS von mind. 1 Standort abgedeckt sein
# Merke: coverage_map enthält nur abdeckbare Kunden und potenzielle Standorte
# Merke: location_vars ist die Solver-Variable, die angibt, ob ein Standort geöffnet wird, kann 0 oder 1 annehmen
for demand_idx, site_idxs in coverage_map.items():
    prob += pulp.lpSum([location_vars[site_idx] for site_idx in site_idxs]) >= 1

# Lösen ohne Ausgabe des Verlaufs
prob.solve(pulp.PULP_CBC_CMD(msg=False))

# ---------------------------------------------------------
# 5. ERGEBNIS AUSWERTUNG
# ---------------------------------------------------------
print("-" * 30)
print(f"Status: {pulp.LpStatus[prob.status]}")
print("-" * 30)

selected_sites = []
# add open locations (value=1) to selected_sites
for site_idx in candidate_indices:
    if location_vars[site_idx].value() == 1.0:
        city_name = df_candidates.at[site_idx, 'city']
        city_plz = df_candidates.at[site_idx, 'plz']
        is_top = df_candidates.at[site_idx, 'is_top_200']
        selected_sites.append(f"{city_name} ({city_plz}) ({'Top-City' if is_top else 'Local'})")

selected_sites.sort()

print(f"Ausgewählte optimale Standorte (Anzahl: {len(selected_sites)}):")
for site in selected_sites:
    print(f"[x] {site}")

print("-" * 30)
print("Interpretation:")
print("Offenbach und Rüsselsheim sollten NICHT gewählt sein, wenn Frankfurt sie abdeckt.")
print("Fulda sollte gewählt sein, da es zu weit weg von Frankfurt ist.")


1. Erstelle Mock-Daten...
2. Geocoding der Standorte...
3. Berechne Distanzen und Coverage-Logik...
-> 0 PLZ-Gebiete können nicht abgedeckt werden (kein Standort in 50.0km).
4. Starte Solver...
------------------------------
Status: Optimal
------------------------------
Ausgewählte optimale Standorte (Anzahl: 2):
[x] Frankfurt am Main (60311) (Top-City)
[x] Fulda (36037) (Local)
------------------------------
Interpretation:
Offenbach und Rüsselsheim sollten NICHT gewählt sein, wenn Frankfurt sie abdeckt.
Fulda sollte gewählt sein, da es zu weit weg von Frankfurt ist.


### Saubere schnellere konfigurierbarere Lösung

Den obigen Code habe ich in Claude gesendet mit der bitte ihn zu optimieren (bessere struktur, einfacher verständlich, gut commentiert und production ready). Nach einigen Iterationen kam das hier raus.

In [ ]:
# =========================================================================
# PURPOSE: Minimize locations to cover a specified amount of customers 
# =========================================================================

import os
import pandas as pd
import numpy as np
import pulp
import pgeocode
import folium
import webbrowser
import logging
from sklearn.metrics.pairwise import haversine_distances

# == CONFIGURATION ========================================================
CONFIG = {
    'max_distance_km': 35.0, # reduced cause real driving time is relevant to customer
    'service_level': 0.90,
    'cost_top_city': 0.8,
    'cost_standard': 1.0,
    'customer_bonus': 0.2,
    'prestige_bonus': 0.1,
    'earth_radius_km': 6371.0,
    'decay_start_km': 10.0,
    'min_weight_at_max': 0.5,
    'candidates_path': 'c:/Users/Thorsten/german_cities_clean_utf8.csv',
    'demand_path': 'c:/Users/Thorsten/90000_customers.csv',
    'results_path': 'c:/Users/Thorsten/optimized_locations_list.csv',
    'log_file': 'c:/Users/Thorsten/optimization_process.log'
}

# == LOGGING ================================================================
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - [%(filename)s] - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(CONFIG['log_file'], mode='a', encoding='utf-8'),
        logging.StreamHandler()
    ]
)

def log_separator():
    try:
        filename = os.path.basename(__file__)
    except NameError:
        filename = "Interactive_Session"
    logging.info("="*60)
    logging.info(f"START: Execution of {filename}")
    logging.info("="*60)

# =============================================================================
# DATA LOADING & PREPARATION
# =============================================================================

def read_data():
    logging.info("Step 1: Reading input CSV files...")
    try:
        df_candidates = pd.read_csv(CONFIG['candidates_path'], dtype={'plz': str})
        threshold = df_candidates['population_total'].nlargest(200).min()
        df_candidates['is_top_200'] = df_candidates['population_total'] >= threshold
        
        df_demand = pd.read_csv(CONFIG['demand_path'])
        if 'plz5' not in df_demand.columns and 'plz-nummer' in df_demand.columns:
            df_demand = df_demand.rename(columns={'plz-nummer': 'plz5'})
        
        df_demand['plz5'] = df_demand['plz5'].astype(str)
        logging.info(f"Loaded {len(df_candidates)} candidates and {len(df_demand)} demand points.")
        return df_candidates, df_demand
    except Exception as e:
        logging.error(f"Failed to read data: {e}")
        raise

def add_coordinates(df, plz_column):
    logging.info(f"Enriching coordinates for {plz_column}...")
    geo = pgeocode.Nominatim('de')
    valid_geo_data = geo._data.dropna(subset=['latitude', 'longitude'])
    valid_zip_set = set(valid_geo_data['postal_code'].unique())

    df[plz_column] = df[plz_column].str.replace('.0', '', regex=False).str.zfill(5)
    initial_count = len(df)
    df = df[df[plz_column].isin(valid_zip_set)].copy()
    
    geo_info = geo.query_postal_code(df[plz_column].tolist())
    df['lat'] = geo_info['latitude'].values
    df['lon'] = geo_info['longitude'].values
    df[['lat_rad', 'lon_rad']] = np.radians(df[['lat', 'lon']])
    
    logging.info(f"Geocoding finished. {len(df)}/{initial_count} locations valid.")
    return df.reset_index(drop=True)

# =============================================================================
# LOGIC & OPTIMIZATION
# =============================================================================

def calculate_coverage(df_demand, df_candidates, max_distance):
    logging.info("Calculating catchment areas and weights...")
    coords_demand = df_demand[['lat_rad', 'lon_rad']].to_numpy()
    coords_candidates = df_candidates[['lat_rad', 'lon_rad']].to_numpy()
    dist_matrix = haversine_distances(coords_demand, coords_candidates) * CONFIG['earth_radius_km']
    
    location_stats = {}
    coverage = {}
    demand_col = 'customer_count'
    max_pop = df_candidates['population_total'].max()

    for s_idx in range(len(df_candidates)):
        c_sum_total = 0
        w_sum_weighted = 0
        reachable_indices = []
        for k_idx in range(len(df_demand)):
            d = dist_matrix[k_idx, s_idx]
            if d <= max_distance:
                reachable_indices.append(k_idx)
                count = df_demand.iloc[k_idx][demand_col]
                if d <= CONFIG['decay_start_km']:
                    weight = 1.0
                else:
                    dist_ratio = (d - CONFIG['decay_start_km']) / (max_distance - CONFIG['decay_start_km'])
                    weight = 1.0 - dist_ratio * (1.0 - CONFIG['min_weight_at_max'])
                c_sum_total += count
                w_sum_weighted += count * weight
        
        loc_id = df_candidates.index[s_idx]
        coverage[loc_id] = reachable_indices
        location_stats[loc_id] = {
            'customers_total': float(c_sum_total),
            'customers_weighted': float(w_sum_weighted),
            'pop_factor': df_candidates.iloc[s_idx]['population_total'] / max_pop
        }

    all_weighted = [s['customers_weighted'] for s in location_stats.values()]
    mx, mn = (max(all_weighted), min(all_weighted)) if all_weighted else (1, 0)
    for loc_id in location_stats:
        location_stats[loc_id]['customer_factor'] = (location_stats[loc_id]['customers_weighted'] - mn) / (mx - mn) if mx > mn else 1.0
            
    cust_to_loc = {k_idx: [loc_id for loc_id, ids in coverage.items() if k_idx in ids] for k_idx in range(len(df_demand))}
    return cust_to_loc, location_stats

def optimize_locations(df_demand, df_candidates, coverage, location_stats):
    logging.info("Starting PuLP optimization...")
    problem = pulp.LpProblem("Location_Optimization", pulp.LpMinimize)
    is_opened = pulp.LpVariable.dicts("loc", df_candidates.index, cat=pulp.LpBinary)
    is_served = pulp.LpVariable.dicts("cust", df_demand.index, cat=pulp.LpBinary)
    
    costs = []
    for i in df_candidates.index:
        base_cost = CONFIG['cost_top_city'] if df_candidates.at[i, 'is_top_200'] else CONFIG['cost_standard']
        bonus = (location_stats[i]['customer_factor'] * CONFIG['customer_bonus']) + \
                (location_stats[i]['pop_factor'] * CONFIG['prestige_bonus'])
        costs.append(is_opened[i] * (base_cost - bonus))
    
    problem += pulp.lpSum(costs)
    for k in df_demand.index:
        problem += pulp.lpSum(is_opened[s] for s in coverage.get(k, [])) >= is_served[k]
    
    demand_col = 'customer_count'
    min_required = df_demand[demand_col].sum() * CONFIG['service_level']
    problem += pulp.lpSum(is_served[i] * df_demand.at[i, demand_col] for i in df_demand.index) >= min_required
    
    problem.solve(pulp.PULP_CBC_CMD(msg=False))
    logging.info(f"Optimization Status: {pulp.LpStatus[problem.status]}")
    return problem, is_opened, is_served # Return problem to check status in main

# =============================================================================
# EXPORT & VISUALIZATION
# =============================================================================

def export_results_to_csv(df_candidates, is_opened, stats, output_path):
    logging.info(f"Generating results CSV: {output_path}")
    opened_indices = [idx for idx in df_candidates.index if is_opened[idx].value() > 0.5]
    
    export_data = []
    for idx in opened_indices:
        row = df_candidates.loc[idx]
        export_data.append({
            'city_name': row['city_name'],
            'plz': row['plz'],
            'city_type': 'Top 200' if row['is_top_200'] else 'Standard',
            'patients_covered_weighted': round(stats[idx]['customers_weighted'], 2),
            'patients_covered_total': int(stats[idx]['customers_total'])
        })
    
    df_results = pd.DataFrame(export_data)
    df_results = df_results.sort_values(by='patients_covered_total', ascending=False)
    df_results.to_csv(output_path, index=False, encoding='utf-8')
    logging.info(f"Export successful. {len(df_results)} locations exported.")

def visualize_and_open(df_candidates, df_demand, is_opened, is_served, stats):
    """
    Creates a focused interactive dashboard:
    - Only opened locations visible.
    - Detailed Pop-ups.
    - Permanent legend with KPIs.
    """
    logging.info("Creating final dashboard map...")
    
    # Initialize map (Centered on Germany)
    m = folium.Map(location=[51.1657, 10.4515], zoom_start=6, control_scale=True)
    
    demand_col = 'customer_count'
    total_customers_data = int(df_demand[demand_col].sum())
    opened_indices = [idx for idx in df_candidates.index if is_opened[idx].value() > 0.5]
    num_opened = len(opened_indices)
    
    covered_customers = sum(
        df_demand.at[idx, demand_col] 
        for idx in df_demand.index 
        if is_served[idx].value() > 0.5
    )

    # 1. Plot opened locations
    for idx in opened_indices:
        row = df_candidates.loc[idx]
        
        popup_html = f"""
        <div style="font-family: Arial; width: 220px;">
            <h4 style="margin-bottom:5px;">{row['city_name']}</h4>
            <hr>
            <b>Status:</b> Opened<br>
            <b>Total Customers in Reach:</b> {stats[idx]['customers_total']:.0f}<br>
            <b>Weighted Potential:</b> {stats[idx]['customers_weighted']:.1f}<br>
            <b>City Type:</b> {'Top 200' if row['is_top_200'] else 'Standard'}
        </div>
        """
        
        folium.Marker(
            [row['lat'], row['lon']], 
            icon=folium.Icon(color='blue', icon='shopping-cart', prefix='fa'), 
            popup=folium.Popup(popup_html, max_width=250)
        ).add_to(m)
        
        # Visualize catchment radius
        folium.Circle(
            [row['lat'], row['lon']], 
            radius=CONFIG['max_distance_km'] * 1000, 
            color='blue', 
            fill=True, 
            fill_opacity=0.1,
            weight=1
        ).add_to(m)

    # 2. Add Permanent Legend (HTML/CSS)
    legend_html = f'''
    <div style="
        position: fixed; 
        bottom: 50px; right: 50px; width: 300px; height: 160px; 
        background-color: white; border:2px solid grey; z-index:9999; font-size:14px;
        padding: 15px; border-radius: 10px; box-shadow: 2px 2px 5px rgba(0,0,0,0.3);
        font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
        ">
        <b style="font-size: 16px;">Location Optimization Dashboard</b><br>
        <hr style="margin: 10px 0;">
        <i class="fa fa-users" style="color:navy"></i> Total Customers: <b>{total_customers_data:,}</b><br>
        <i class="fa fa-map-marker" style="color:blue"></i> Opened Locations: <b>{num_opened}</b><br>
        <i class="fa fa-check-circle" style="color:green"></i> Covered Customers: <b>{int(covered_customers):,}</b><br>
        <i class="fa fa-pie-chart" style="color:orange"></i> Actual Service Level: <b>{(covered_customers/total_customers_data)*100:.1f}%</b>
    </div>
    '''
    m.get_root().html.add_child(folium.Element(legend_html))

    # 3. Save and Open
    user_home = os.path.expanduser("~")
    map_path = os.path.join(user_home, "location_optimization_dashboard.html")
    m.save(map_path)
    
    logging.info(f"Dashboard saved: {map_path}")
    logging.info(f"Result: {num_opened} locations cover {int(covered_customers)} customers.")
    webbrowser.open('file://' + os.path.realpath(map_path))

# =============================================================================
# MAIN EXECUTION
# =============================================================================

def main():
    # 1. Logging start
    log_separator()
    logging.info("--- Optimization Process Started ---")
    
    # 2. Data loading an enrichment
    df_candidates, df_demand = read_data()
    df_candidates = add_coordinates(df_candidates, 'plz')
    df_demand = add_coordinates(df_demand, 'plz5')
    
    # 3. Coverage calculation and optimization
    coverage, stats = calculate_coverage(df_demand, df_candidates, CONFIG['max_distance_km'])
    # ERROR FIX: We now receive the problem object to check its status properly
    problem, is_opened, is_served = optimize_locations(df_demand, df_candidates, coverage, stats)

    # 4. Export and Visualization only if optimal solution found
    if pulp.LpStatus[problem.status] == 'Optimal':
        visualize_and_open(df_candidates, df_demand, is_opened, is_served, stats)
        export_results_to_csv(df_candidates, is_opened, stats, CONFIG['results_path'])
    else:
        logging.error(f"Solution status: {pulp.LpStatus[problem.status]}. Export and Visualization skipped.")
    
    # 5. Logging completion
    logging.info("--- Optimization Process Finished Successfully ---")

if __name__ == "__main__":
    main()


Status: Optimal

Gewählte Standorte:
 - Frankfurt am Main 		(Pop: 764,000, Kunden (gewichtet): 2300.0 (2218.7))
 - München 		(Pop: 1,488,000, Kunden (gewichtet): 1200.0 (1200.0))
 - Hamburg 		(Pop: 1,853,000, Kunden (gewichtet): 1100.0 (1100.0))
 - Berlin 		(Pop: 3,664,000, Kunden (gewichtet): 1500.0 (1500.0))


## Distanzberechnungen

Wie funktionieren sie, welche Unterschiede gibt es.

### Standard Haversine Berechnung: 1-1 Distanz

Die Bibliothek erwartet Koordinaten als Tupel im Format (Breitengrad, Längengrad). Mit der Standard Haversine Funktion kann man stets nur Distanz von zwei Punkten berechnen.

In [5]:
from haversine import haversine, Unit

# Beispiel-Koordinaten (Latitude, Longitude)
# Ersetze diese Variablen mit deinen Eingabedaten
start_punkt = (52.5200, 13.4050) # Berlin
end_punkt = (48.1351, 11.5820)   # München

# Berechnung der Distanz
# unit=Unit.KILOMETERS ist Standard, kann aber auch auf MILES, METERS etc. geändert werden
distanz = haversine(start_punkt, end_punkt, unit=Unit.KILOMETERS)

print(f"Die Distanz beträgt {distanz:.2f} km")


Die Distanz beträgt 504.42 km


### Paarweise Berechnung von Listen

Für zwei Listen mit Start und Zielpunkten eignet sich die vectorisierte Haversine Funktion. Diese ist ein vielfaches Schneller und erlaubt es Listen als Argumente zu übergeben.

In [9]:
from haversine import haversine_vector, Unit

# Liste von Start-Koordinaten (Lat, Lon)
start_punkte = [
    (52.5200, 13.4050), # Berlin
    (48.8566, 2.3522)   # Paris
]

# Liste von Ziel-Koordinaten (Lat, Lon)
end_punkte = [
    (48.1351, 11.5820), # München
    (51.5074, -0.1278)  # London
]

# Berechnet Distanz: Berlin->München UND Paris->London
distanzen = haversine_vector(start_punkte, end_punkte, unit=Unit.KILOMETERS)

print(distanzen)
# Ausgabe (Beispiel): [504.23, 343.56]


[504.41602813 343.55653488]


### Ein Startpunkt mit vielen Zielen

Wenn du wissen willst, wie weit ein Standort von einer Liste anderer Standorte entfernt ist, kannst du den comb (Combination) Parameter nutzen oder die Listen entsprechend vorbereiten. Die haversine_vector Funktion ist hier sehr flexibel.

Am einfachsten ist es oft, den Startpunkt in eine Liste zu packen, die so lang ist wie die Zielliste (Broadcasting), oder direkt Pandas zu nutzen, falls du DataFrames verwendest.

### Schnelle Alternative für viele Datenpunkte

Falls du mit vielen Datenpunkten oder Arrays arbeitest (z. B. in Pandas oder NumPy), ist scikit-learn oft effizienter, da es vektorisierte Operationen unterstützt. Beachte, dass scikit-learn Bogenmaß (Radians) erwartet.

In [7]:
from sklearn.metrics.pairwise import haversine_distances
from math import radians

# Koordinaten in Radians umwandeln
start_in_radians = [radians(52.5200), radians(13.4050)]
end_in_radians = [radians(48.1351), radians(11.5820)]

# Ergebnis ist in Radians, muss mit Erdradius multipliziert werden (ca. 6371 km)
result = haversine_distances([start_in_radians], [end_in_radians])
distanz_km = result[0][0] * 6371  # Radius in km

print(f"Die Distanz beträgt {distanz_km:.2f} km")


Die Distanz beträgt 504.42 km


### Erstellen einer Distanzmatrix

Zwei Methoden:
1. haversine_vector: das Argument comb=True und Übergabe einer Standorteliste als Start- und als Zielpunktliste. 
2. skikit-learn: Für viele Standorte ist das einfacher schenller


In [3]:
import pandas as pd
from haversine import haversine_vector, Unit

# 1. Liste der Standorte (Lat, Lon)
standorte = [
    (52.5200, 13.4050), # Berlin
    (48.1351, 11.5820), # München
    (53.5511, 9.9937),  # Hamburg
    (50.1109, 8.6821)   # Frankfurt
]

# Namen für die bessere Lesbarkeit (optional)
namen = ["Berlin", "München", "Hamburg", "Frankfurt"]

# 2. Berechnung der Matrix
# comb=True sorgt dafür, dass jeder Punkt aus Liste 1 mit jedem aus Liste 2 verglichen wird.
distanz_matrix = haversine_vector(standorte, standorte, unit=Unit.KILOMETERS, comb=True)

# 3. Darstellung als DataFrame
df_matrix = pd.DataFrame(distanz_matrix, columns=namen, index=namen)

print("Distanzmatrix (in km):")
print(df_matrix)


Distanzmatrix (in km):
               Berlin     München     Hamburg   Frankfurt
Berlin       0.000000  504.416028  255.250519  423.528610
München    504.416028    0.000000  612.428927  304.585539
Hamburg    255.250519  612.428927    0.000000  392.989090
Frankfurt  423.528610  304.585539  392.989090    0.000000


In [4]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import haversine_distances
from math import radians

# Standorte (Lat, Lon)
standorte = [
    (52.5200, 13.4050), # Berlin
    (48.1351, 11.5820), # München
    (53.5511, 9.9937),  # Hamburg
    (50.1109, 8.6821)   # Frankfurt
]
namen = ["Berlin", "München", "Hamburg", "Frankfurt"]

# 1. Umwandlung in Radians (Scikit-Learn erwartet Bogenmaß!)
# Wir nutzen np.radians für die Vektorisierung, falls standorte schon ein Array wäre,
# hier nutzen wir eine List Comprehension für reine Python Listen:
standorte_rad = [[radians(lat), radians(lon)] for lat, lon in standorte]

# 2. Berechnung der Distanzen (Ergebnis ist in Radians)
matrix_rad = haversine_distances(standorte_rad, standorte_rad)

# 3. Umrechnung zurück in Kilometer (Erdradius ca. 6371 km)
matrix_km = matrix_rad * 6371

# 4. Schöner DataFrame
df_matrix = pd.DataFrame(matrix_km, columns=namen, index=namen)

# Runden für schönere Ausgabe
print(df_matrix.round(2))


           Berlin  München  Hamburg  Frankfurt
Berlin       0.00   504.42   255.25     423.53
München    504.42     0.00   612.43     304.59
Hamburg    255.25   612.43     0.00     392.99
Frankfurt  423.53   304.59   392.99       0.00
